<a href="https://colab.research.google.com/github/MMoltira/Chest-X-Ray-classification/blob/main/Pretrained_Sampling_Ab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Pretrained // Sampling Abnormal K-fold

2) 0-100 // Sampling Abnormal K-fold

#### import

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import PIL
from PIL import Image


In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from imutils import paths
import argparse

from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore


import matplotlib.image as mpimg
from matplotlib.pyplot import savefig

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

#### Load data // KKU

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_dir = '/content/drive/MyDrive/Senior Project'

train 3 class

In [5]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

#### Load data // Gmail

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
data_dir = '/content/drive/MyDrive/Senior Project'

train 3 class

In [8]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

#### Data 2 class  + Sampling data
ข้อมูลใน All_3class_dir รวมภาพทั้ง 3 คลาส รวม 451 ภาพ TB150, Normal151 และ CA150 (CA คือ lungcancer)

In [9]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

In [10]:
normal = []
for i in range(len(os.listdir(Normal151_dir))):
    img = os.listdir(Normal151_dir)[i]
    normal.append(img)
    
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal"  })

In [11]:
tuberculosis = []
for i in range(len(os.listdir(TB150_dir))):
    img = os.listdir(TB150_dir)[i]
    tuberculosis.append(img)

ttuberculosis = pd.DataFrame({'ImageName':[tuberculosis][0],
                                       'NameType': "Abnormal"  })

In [12]:
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Abnormal"  })

In [13]:
data_train3 = pd.concat([tnormal[:-1], ttuberculosis, tlungcancer])
data_train3[-3:]

,ImageName,NameType
147,ca94.jpg,Abnormal
148,ca85.jpg,Abnormal
149,ca87.jpg,Abnormal


In [14]:
data_train3.shape

(450, 2)

Sampling data

In [15]:
tabnormal= pd.concat([ttuberculosis, tlungcancer])
tabnormal = tabnormal.sample(n=150, random_state=40)
data_train3 = pd.concat([tnormal[:-1], tabnormal])

In [16]:
data_train3.to_excel("data_2class_sampling.xlsx")

# Pretrained Sampling Ab

- sigmold
- binary_crossentropy

### 1

#### 1 sig pre bi
--> 

In [ ]:
BS = 1
filepath = "Kmodel1_sig_pre_bi"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [02:32<00:00,  1.97it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
239/240 [============================>.] - ETA: 0s - loss: 0.6564 - accuracy: 0.5858
Epoch 1: val_accuracy improved from -inf to 0.65000, saving model to  Kmodel1_sig_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_sig_pre_bi1.h5 /assets
240/240 [==============================] - 65s 210ms/step - loss: 0.6565 - accuracy: 0.5875 - val_loss: 0.6444 - val_accuracy: 0.6500 - lr: 1.0000e-04
Epoch 2/100
240/240 [==============================] - ETA: 0s - loss: 0.5913 - accuracy: 0.7625
Epoch 2: val_accuracy improved from 0.65000 to 0.70000, saving model to  Kmodel1_sig_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_sig_pre_bi1.h5 /assets
240/240 [==============================] - 43s 180ms/step - loss: 0.5913 - accuracy: 0.7625 - val_loss: 0.6064 - val_accuracy: 0.7000 - lr: 1.0000e-04
Epoch 3/100
240/240 [==============================] - ETA: 0s - loss: 0.5541 - accuracy: 0.8250
Epoch 3: 

#### 1 sig pre bi adam
--> 

In [ ]:
BS = 1
filepath = "Kmodel1_sig_pre_bi_adam"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:02<00:00, 100.38it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
239/240 [============================>.] - ETA: 0s - loss: 0.6832 - accuracy: 0.5816
Epoch 1: val_accuracy improved from -inf to 0.70000, saving model to  Kmodel1_sig_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_sig_pre_bi_adam1.h5 /assets
240/240 [==============================] - 62s 222ms/step - loss: 0.6834 - accuracy: 0.5792 - val_loss: 0.6356 - val_accuracy: 0.7000 - lr: 1.0000e-04
Epoch 2/100
239/240 [============================>.] - ETA: 0s - loss: 0.6260 - accuracy: 0.6736
Epoch 2: val_accuracy improved from 0.70000 to 0.76667, saving model to  Kmodel1_sig_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_sig_pre_bi_adam1.h5 /assets
240/240 [==============================] - 42s 177ms/step - loss: 0.6258 - accuracy: 0.6750 - val_loss: 0.5917 - val_accuracy: 0.7667 - lr: 1.0000e-04
Epoch 3/100
239/240 [============================>.] - ETA: 0s - loss: 0.5747 - accuracy: 0.7406
Epoch 3: val_accuracy did not improve from 0.76667
240

#### 1 sig pre bi sgd
--> 

In [ ]:
BS = 1
filepath = "Kmodel1_sig_pre_bi_sgd"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:03<00:00, 95.58it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
240/240 [==============================] - ETA: 0s - loss: 0.6992 - accuracy: 0.5458
Epoch 1: val_accuracy improved from -inf to 0.65000, saving model to  Kmodel1_sig_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_sig_pre_bi_sgd1.h5 /assets
240/240 [==============================] - 61s 221ms/step - loss: 0.6992 - accuracy: 0.5458 - val_loss: 0.6601 - val_accuracy: 0.6500 - lr: 1.0000e-04
Epoch 2/100
240/240 [==============================] - ETA: 0s - loss: 0.6232 - accuracy: 0.7042
Epoch 2: val_accuracy improved from 0.65000 to 0.78333, saving model to  Kmodel1_sig_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_sig_pre_bi_sgd1.h5 /assets
240/240 [==============================] - 43s 179ms/step - loss: 0.6232 - accuracy: 0.7042 - val_loss: 0.6158 - val_accuracy: 0.7833 - lr: 1.0000e-04
Epoch 3/100
240/240 [==============================] - ETA: 0s - loss: 0.5706 - accuracy: 0.7625
Epoch 3: val_accuracy improved from 0.78333 to 0.80000, sa

### 8

#### 8  sig pre bi
--> 

In [ ]:
BS = 8
filepath = "Kmodel8_sig_pre_bi"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:03<00:00, 94.57it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
30/30 [==============================] - ETA: 0s - loss: 0.7352 - accuracy: 0.4083
Epoch 1: val_accuracy improved from -inf to 0.51667, saving model to  Kmodel8_sig_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_sig_pre_bi1.h5 /assets
30/30 [==============================] - 57s 2s/step - loss: 0.7352 - accuracy: 0.4083 - val_loss: 0.7091 - val_accuracy: 0.5167 - lr: 1.0000e-04
Epoch 2/100
30/30 [==============================] - ETA: 0s - loss: 0.7049 - accuracy: 0.5292
Epoch 2: val_accuracy did not improve from 0.51667
30/30 [==============================] - 6s 209ms/step - loss: 0.7049 - accuracy: 0.5292 - val_loss: 0.7099 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 3/100
30/30 [==============================] - ETA: 0s - loss: 0.6796 - accuracy: 0.5917
Epoch 3: val_accuracy improved from 0.51667 to 0.65000, saving model to  Kmodel8_sig_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_sig_pre_bi1.h5 /assets
30/30 [==============================

#### 8 sig pre bi adam
--> 

In [ ]:
BS = 8
filepath = "Kmodel8_sig_pre_bi_adam"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:03<00:00, 96.69it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
30/30 [==============================] - ETA: 0s - loss: 0.7212 - accuracy: 0.4792
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to  Kmodel8_sig_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_sig_pre_bi_adam1.h5 /assets
30/30 [==============================] - 58s 2s/step - loss: 0.7212 - accuracy: 0.4792 - val_loss: 0.7154 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 2/100
30/30 [==============================] - ETA: 0s - loss: 0.6874 - accuracy: 0.5042
Epoch 2: val_accuracy improved from 0.50000 to 0.55000, saving model to  Kmodel8_sig_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_sig_pre_bi_adam1.h5 /assets
30/30 [==============================] - 39s 1s/step - loss: 0.6874 - accuracy: 0.5042 - val_loss: 0.6922 - val_accuracy: 0.5500 - lr: 1.0000e-04
Epoch 3/100
30/30 [==============================] - ETA: 0s - loss: 0.6731 - accuracy: 0.5708
Epoch 3: val_accuracy improved from 0.55000 to 0.65000, saving model t

#### 8 sig pre bi sgd
--> 

In [ ]:
BS = 8
filepath = "Kmodel8_sig_pre_bi_sgd"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:03<00:00, 97.10it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
30/30 [==============================] - ETA: 0s - loss: 0.7105 - accuracy: 0.5417
Epoch 1: val_accuracy improved from -inf to 0.40000, saving model to  Kmodel8_sig_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_sig_pre_bi_sgd1.h5 /assets
30/30 [==============================] - 59s 2s/step - loss: 0.7105 - accuracy: 0.5417 - val_loss: 0.7320 - val_accuracy: 0.4000 - lr: 1.0000e-04
Epoch 2/100
30/30 [==============================] - ETA: 0s - loss: 0.6898 - accuracy: 0.5417
Epoch 2: val_accuracy improved from 0.40000 to 0.43333, saving model to  Kmodel8_sig_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_sig_pre_bi_sgd1.h5 /assets
30/30 [==============================] - 40s 1s/step - loss: 0.6898 - accuracy: 0.5417 - val_loss: 0.7173 - val_accuracy: 0.4333 - lr: 1.0000e-04
Epoch 3/100
30/30 [==============================] - ETA: 0s - loss: 0.6667 - accuracy: 0.6333
Epoch 3: val_accuracy improved from 0.43333 to 0.56667, saving model to  K

### 16

#### 16  sig pre bi
--> 

In [17]:
BS = 16
filepath = "Kmodel16_sig_pre_bi"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [02:38<00:00,  1.90it/s]


In [18]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

15/15 [==============================] - ETA: 0s - loss: 0.6725 - accuracy: 0.6250
Epoch 1: val_accuracy improved from -inf to 0.66667, saving model to  Kmodel16_sig_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_sig_pre_bi1.h5 /assets
15/15 [==============================] - 61s 3s/step - loss: 0.6725 - accuracy: 0.6250 - val_loss: 0.6550 - val_accuracy: 0.6667 - lr: 1.0000e-04
Epoch 2/100
15/15 [==============================] - ETA: 0s - loss: 0.6497 - accuracy: 0.6542
Epoch 2: val_accuracy improved from 0.66667 to 0.71667, saving model to  Kmodel16_sig_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_sig_pre_bi1.h5 /assets
15/15 [==============================] - 39s 3s/step - loss: 0.6497 - accuracy: 0.6542 - val_loss: 0.6416 - val_accuracy: 0.7167 - lr: 1.0000e-04
Epoch 3/100
15/15 [==============================] - ETA: 0s - loss: 0.6450 - accuracy: 0.6917
Epoch 3: val_accuracy improved from 0.71667 to 0.78333, saving model to  Kmodel16_sig_pre_bi1.h5 
INFO:ten

#### 16 sig pre bi adam
--> 

In [19]:
BS = 16
filepath = "Kmodel16_sig_pre_bi_adam"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:04<00:00, 64.89it/s]


In [20]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
15/15 [==============================] - ETA: 0s - loss: 0.7107 - accuracy: 0.5542
Epoch 1: val_accuracy improved from -inf to 0.53333, saving model to  Kmodel16_sig_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_sig_pre_bi_adam1.h5 /assets
15/15 [==============================] - 59s 4s/step - loss: 0.7107 - accuracy: 0.5542 - val_loss: 0.7092 - val_accuracy: 0.5333 - lr: 1.0000e-04
Epoch 2/100
15/15 [==============================] - ETA: 0s - loss: 0.6726 - accuracy: 0.6167
Epoch 2: val_accuracy improved from 0.53333 to 0.61667, saving model to  Kmodel16_sig_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_sig_pre_bi_adam1.h5 /assets
15/15 [==============================] - 39s 3s/step - loss: 0.6726 - accuracy: 0.6167 - val_loss: 0.6852 - val_accuracy: 0.6167 - lr: 1.0000e-04
Epoch 3/100
15/15 [==============================] - ETA: 0s - loss: 0.6627 - accuracy: 0.6208
Epoch 3: val_accuracy improved from 0.61667 to 0.63333, saving mod

#### 16 sig pre bi sgd
--> 

In [21]:
BS = 16
filepath = "Kmodel16_sig_pre_bi_sgd"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:04<00:00, 74.15it/s]


In [22]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
15/15 [==============================] - ETA: 0s - loss: 0.7164 - accuracy: 0.5375
Epoch 1: val_accuracy improved from -inf to 0.53333, saving model to  Kmodel16_sig_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_sig_pre_bi_sgd1.h5 /assets
15/15 [==============================] - 64s 4s/step - loss: 0.7164 - accuracy: 0.5375 - val_loss: 0.7001 - val_accuracy: 0.5333 - lr: 1.0000e-04
Epoch 2/100
15/15 [==============================] - ETA: 0s - loss: 0.6868 - accuracy: 0.5542
Epoch 2: val_accuracy did not improve from 0.53333
15/15 [==============================] - 6s 399ms/step - loss: 0.6868 - accuracy: 0.5542 - val_loss: 0.6935 - val_accuracy: 0.5167 - lr: 1.0000e-04
Epoch 3/100
15/15 [==============================] - ETA: 0s - loss: 0.6745 - accuracy: 0.6042
Epoch 3: val_accuracy improved from 0.53333 to 0.65000, saving model to  Kmodel16_sig_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_sig_pre_bi_sgd1.h5 /assets
15/15 [==========

### 32

#### 32  sig pre bi
--> 

In [23]:
BS = 32
filepath = "Kmodel32_sig_pre_bi"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:03<00:00, 96.44it/s] 


In [24]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
7/7 [==============================] - ETA: 0s - loss: 0.7058 - accuracy: 0.4615
Epoch 1: val_accuracy improved from -inf to 0.40000, saving model to  Kmodel32_sig_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_sig_pre_bi1.h5 /assets
7/7 [==============================] - 57s 8s/step - loss: 0.7058 - accuracy: 0.4615 - val_loss: 0.7438 - val_accuracy: 0.4000 - lr: 1.0000e-04
Epoch 2/100
7/7 [==============================] - ETA: 0s - loss: 0.6998 - accuracy: 0.5337
Epoch 2: val_accuracy improved from 0.40000 to 0.41667, saving model to  Kmodel32_sig_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_sig_pre_bi1.h5 /assets
7/7 [==============================] - 38s 6s/step - loss: 0.6998 - accuracy: 0.5337 - val_loss: 0.7269 - val_accuracy: 0.4167 - lr: 1.0000e-04
Epoch 3/100
7/7 [==============================] - ETA: 0s - loss: 0.6783 - accuracy: 0.5865
Epoch 3: val_accuracy did not improve from 0.41667
7/7 [==============================] - 5s 725

#### 32 sig pre bi adam
--> 

In [25]:
BS = 32
filepath = "Kmodel32_sig_pre_bi_adam"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:03<00:00, 92.54it/s]


In [26]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
7/7 [==============================] - ETA: 0s - loss: 0.7797 - accuracy: 0.4904
Epoch 1: val_accuracy improved from -inf to 0.48333, saving model to  Kmodel32_sig_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_sig_pre_bi_adam1.h5 /assets
7/7 [==============================] - 61s 8s/step - loss: 0.7797 - accuracy: 0.4904 - val_loss: 0.7375 - val_accuracy: 0.4833 - lr: 1.0000e-04
Epoch 2/100
7/7 [==============================] - ETA: 0s - loss: 0.7432 - accuracy: 0.4087
Epoch 2: val_accuracy did not improve from 0.48333
7/7 [==============================] - ETA: 0s - loss: 0.7217 - accuracy: 0.4327
Epoch 3: val_accuracy did not improve from 0.48333
7/7 [==============================] - 5s 724ms/step - loss: 0.7217 - accuracy: 0.4327 - val_loss: 0.7133 - val_accuracy: 0.4833 - lr: 1.0000e-04
Epoch 4/100
7/7 [==============================] - ETA: 0s - loss: 0.7096 - accuracy: 0.4615
Epoch 4: val_accuracy did not improve from 0.48333
7/7 [===========

#### 32 sig pre bi sgd
--> 

In [27]:
BS = 32
filepath = "Kmodel32_sig_pre_bi_sgd"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:03<00:00, 90.81it/s]


In [28]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
7/7 [==============================] - ETA: 0s - loss: 0.6890 - accuracy: 0.5577
Epoch 1: val_accuracy improved from -inf to 0.68333, saving model to  Kmodel32_sig_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_sig_pre_bi_sgd1.h5 /assets
7/7 [==============================] - 58s 8s/step - loss: 0.6890 - accuracy: 0.5577 - val_loss: 0.6499 - val_accuracy: 0.6833 - lr: 1.0000e-04
Epoch 2/100
7/7 [==============================] - ETA: 0s - loss: 0.6645 - accuracy: 0.6394
Epoch 2: val_accuracy improved from 0.68333 to 0.71667, saving model to  Kmodel32_sig_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_sig_pre_bi_sgd1.h5 /assets
7/7 [==============================] - 38s 6s/step - loss: 0.6645 - accuracy: 0.6394 - val_loss: 0.6468 - val_accuracy: 0.7167 - lr: 1.0000e-04
Epoch 3/100
7/7 [==============================] - ETA: 0s - loss: 0.6587 - accuracy: 0.6346
Epoch 3: val_accuracy did not improve from 0.71667
7/7 [========================

# 0-100 Sampling Ab
- sigmold
- binary_crossentropy

### 1

#### 1 sig pre bi
--> 

In [ ]:
BS = 1
filepath = "Kmodel1_sig_pre_bi"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [00:10<00:00, 44.85it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

#### 1 sig pre bi adam
--> 

In [ ]:
BS = 1
filepath = "Kmodel1_sig_pre_bi_adam"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [02:08<00:00,  3.50it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

#### 1 sig pre bi sgd
--> 

In [ ]:
BS = 1
filepath = "Kmodel1_sig_pre_bi_sgd"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [02:08<00:00,  3.50it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

### 8

#### 8  sig pre bi
--> 

In [ ]:
BS = 8
filepath = "Kmodel8_sig_pre_bi"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [00:10<00:00, 44.85it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

#### 8 sig pre bi adam
--> 

In [ ]:
BS = 8
filepath = "Kmodel8_sig_pre_bi_adam"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [02:08<00:00,  3.50it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

#### 8 sig pre bi sgd
--> 

In [ ]:
BS = 8
filepath = "Kmodel8_sig_pre_bi_sgd"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [02:08<00:00,  3.50it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

### 16

#### 16  sig pre bi
--> 

In [ ]:
BS = 16
filepath = "Kmodel16_sig_pre_bi"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [00:10<00:00, 44.85it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

#### 16 sig pre bi adam
--> 

In [ ]:
BS = 16
filepath = "Kmodel16_sig_pre_bi_adam"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [02:08<00:00,  3.50it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

#### 16 sig pre bi sgd
--> 

In [ ]:
BS = 16
filepath = "Kmodel16_sig_pre_bi_sgd"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [02:08<00:00,  3.50it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

### 32

#### 32  sig pre bi
--> 

In [ ]:
BS = 32
filepath = "Kmodel32_sig_pre_bi"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [00:10<00:00, 44.85it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

#### 32 sig pre bi adam
--> 

In [ ]:
BS = 32
filepath = "Kmodel32_sig_pre_bi_adam"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 451/451 [02:08<00:00,  3.50it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmold")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

#### 32 sig pre bi sgd
--> 

In [29]:
BS = 32
filepath = "Kmodel32_sig_pre_bi_sgd"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=2)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                          
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 300/300 [00:03<00:00, 87.18it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # unit 2
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" {filepath}{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
7/7 [==============================] - ETA: 0s - loss: 0.4199 - accuracy: 0.7981
Epoch 1: val_accuracy improved from -inf to 0.60000, saving model to  Kmodel32_sig_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_sig_pre_bi_sgd1.h5 /assets
7/7 [==============================] - 68s 8s/step - loss: 0.4199 - accuracy: 0.7981 - val_loss: 1.0439 - val_accuracy: 0.6000 - lr: 1.0000e-04
Epoch 2/100
7/7 [==============================] - ETA: 0s - loss: 0.1819 - accuracy: 0.9471
Epoch 2: val_accuracy did not improve from 0.60000
7/7 [==============================] - 6s 935ms/step - loss: 0.1819 - accuracy: 0.9471 - val_loss: 1.5980 - val_accuracy: 0.6000 - lr: 1.0000e-04
Epoch 3/100
7/7 [==============================] - ETA: 0s - loss: 0.1036 - accuracy: 0.9663
Epoch 3: val_accuracy did not improve from 0.60000
7/7 [==============================] - 7s 883ms/step - loss: 0.1036 - accuracy: 0.9663 - val_loss: 1.7720 - val_accuracy: 0.5833 - lr: 1.0000e-04
Epoc